# 🧠 Functional Connectivity – Alpha Band

This notebook analyzes the functional connectivity network for the **alpha band**.

---

### 🔄 Modifying for Other Frequency Bands

To analyze other frequency bands (like theta, beta, or gamma), simply update the band-pass filter parameters or frequency selection step in the preprocessing section. The rest of the analysis remains unchanged.

```python
# Example for theta band
low_freq = 4
high_freq = 8
```

Make sure to reflect this change in any plots or network interpretation steps.


In [ ]:
import numpy as np
import random

In [ ]:
import mne
import matplotlib.pyplot as plt
from mne.datasets import sample
from mne_connectivity import spectral_connectivity_epochs

In [ ]:
import networkx as nx

In [ ]:
data_path = sample.data_path()
raw_fname = data_path / 'MEG/sample/sample_audvis_filt-0-40_raw.fif'
event_fname = data_path / 'MEG/sample/sample_audvis_filt-0-40_raw-eve.fif'
raw = mne.io.read_raw_fif(raw_fname)
events = mne.read_events(event_fname)


In [ ]:
print(raw)
print(raw.info)

In [ ]:
raw.pick_types(eeg=True)

In [ ]:
raw.plot(n_channels=59)

In [ ]:
raw.load_data()

In [ ]:
# raw.filter(0.5, 4, fir_design='firwin')

In [ ]:
ica = mne.preprocessing.ICA(n_components=59, random_state=42)
ica.fit(raw)
ica.apply(raw)

In [ ]:
epochs = mne.Epochs(raw,events)

In [ ]:
sfreq = 150.15
tmin= 0
fmin= 8
fmax= 13

#alpha band

In [ ]:
con_pli = spectral_connectivity_epochs(
    epochs, method='pli', mode='multitaper', sfreq=sfreq, fmin=fmin,
    fmax=fmax, faverage=True, tmin=tmin, mt_adaptive=False, n_jobs=1)


In [ ]:
np.shape(con_pli)

In [ ]:
np.shape(con_pli.get_data('dense'))

In [ ]:
conn=con_pli.get_data('dense')
connectivity=conn[:,:,0]

In [ ]:
np.shape(connectivity)

In [ ]:
mat = connectivity
# Get the lower triangle indices
lower_triangle_indices = np.tril_indices_from(mat, k=-1)

# Get the values below the diagonal
lower_triangle_values = mat[lower_triangle_indices]

# Mirror the values onto the values above the diagonal
mat[np.triu_indices_from(mat, k=1)] = lower_triangle_values

# Print the resulting matrix with mirrored values
print(mat)

In [ ]:
is_symmetric = np.array_equal(mat, mat.T)

# Print the result
if is_symmetric:
    print("The matrix is symmetric.")
else:
    print("The matrix is not symmetric.")

In [ ]:
# Create a figure and axis
fig, ax = plt.subplots()

# Create a heatmap using matshow
heatmap = ax.matshow(mat, cmap='viridis')

# Add a colorbar
cbar = plt.colorbar(heatmap)

# Add title and labels
ax.set_title('Functional Connectivity Matrix')
ax.set_xlabel('Nodes')
ax.set_ylabel('Nodes')

# Show the plot
plt.show()


In [ ]:
# Set a threshold value
threshold = 0.05  # Adjust the threshold as desired

# Apply thresholding to convert to a binary matrix
binary_matrix = np.where(mat >= threshold, 1, 0)

# Print the binary matrix
print(binary_matrix)

plt.imshow(binary_matrix, cmap='hot', interpolation='nearest')

# Add colorbar
plt.colorbar()

# Set labels and title
plt.xlabel('Nodes')
plt.ylabel('Nodes')
plt.title('binary Matrix')

# Display the plot
plt.show()

In [ ]:
mat1 = binary_matrix
# Create an empty graph
graph = nx.Graph()

# Add nodes to the graph
num_nodes = mat1.shape[0]
graph.add_nodes_from(range(num_nodes))

# Add edges to the graph based on the binary connectivity matrix
rows, cols = np.where(mat1 == 1)
edges = zip(rows, cols)
graph.add_edges_from(edges)

# Visualize the graph
nx.draw(graph, with_labels=True)
plt.show()

In [ ]:
# Calculate the clustering coefficient
Cp = nx.average_clustering(graph)

# Calculate the characteristic path length
Lp = nx.average_shortest_path_length(graph)

# Print the results
print("Clustering Coefficient (C):", Cp)
print("Characteristic Path Length(L):", Lp)


In [ ]:
# Create a surrogate graph with the same structure as the binary graph
surrogate_graph = nx.Graph()
surrogate_graph.add_nodes_from(graph.nodes)
surrogate_graph.add_edges_from(graph.edges)

# Shuffle the weights of the edges in the surrogate graph
edges = list(surrogate_graph.edges)
weights = np.random.permutation(len(edges))
for i, (u, v) in enumerate(edges):
    surrogate_graph[u][v]['weight'] = weights[i]

# Visualize the serogate graph
nx.draw(surrogate_graph, with_labels=True)
plt.show()

In [ ]:
grap=surrogate_graph
# Calculate the clustering coefficient
Cps = nx.average_clustering(grap)

# Calculate the characteristic path length
Lps = nx.average_shortest_path_length(grap)

# Print the results
print("Clustering Coefficient (Cps):", Cps)
print("Characteristic Path Length(Lps):", Lps)


In [ ]:
#normalized variable

gamma = Cp/Cps #normalised clustering coefficient.
lamda = Lp/Lps #normalised characteristic path length.
sigma = gamma/lamda

In [ ]:
gamma

In [ ]:
lamda

In [ ]:
sigma

In [ ]:
G = graph

# Add nodes and edges to the graph

# Calculate the average clustering coefficient of the network
clustering_coefficient = nx.average_clustering(G)

# Calculate the average shortest path length of the network
characteristic_path_length = nx.average_shortest_path_length(G)

# Generate an equivalent random network with the same number of nodes and edges
random_graph = nx.random_reference(G, niter=10)

# Calculate the average clustering coefficient of the random network
random_clustering_coefficient = nx.average_clustering(random_graph)

# Calculate the average shortest path length of the random network
random_characteristic_path_length = nx.average_shortest_path_length(random_graph)

# Calculate the small-worldness coefficient
small_worldness = (clustering_coefficient / random_clustering_coefficient) / (characteristic_path_length / random_characteristic_path_length)

print("Small-Worldness Coefficient:", small_worldness)
print("Cp =",clustering_coefficient,'\n Lp=',characteristic_path_length,'\n Cps=',random_clustering_coefficient,'Lps=',random_characteristic_path_length)

In [ ]:
gam = clustering_coefficient/characteristic_path_length #normalised clustering coefficient.
lam = random_clustering_coefficient/random_characteristic_path_length #normalised characteristic path length.
sig = gam/lam
print('gam:',gam,'\n lam:',lam,'\n sig:',sig)